In [ ]:
from transformers import  AutoTokenizer, OPTForCausalLM, set_seed, AutoModel
import torch
import time
from prompt import build_prompt, get_prompt_templates, get_prompt_templates_withtype, build_prompt_on_type, get_prompt_templates_industry, build_prompt_on_industry
from utils import filter_unfinished_output, filter_length_token, filter_length_sentence, select_selling_point, first_token_upper
from tqdm import tqdm
import random
import pdb


bytedrive = '/mnt/bn/lq-lby-bytenas-aigc/opt_model/'
config_path = 'config_30B'
device = 'cuda'
templates_path = '/mnt/bn/lq-lby-bytenas-aigc/hook_generation/data/templates.txt'
templates_withtype_path = '/mnt/bn/lq-lby-bytenas-aigc/hook_generation/data/templates_withtype.txt'
templates_sp_path = '/mnt/bn/lq-lby-bytenas-aigc/hook_generation/data/hook_templates_product_sps_indu_hook_type.txt'

s_time = time.time()
tokenizer = AutoTokenizer.from_pretrained(bytedrive + config_path, use_fast=False)
model = OPTForCausalLM.from_pretrained(bytedrive + config_path, torch_dtype=torch.float16, device_map='auto')
print('======loading model used time:', time.time() - s_time)

In [ ]:
import os
os.system('nvidia-smi')

In [ ]:
from prompt import build_prompt, get_prompt_templates, get_prompt_templates_withtype, build_prompt_on_type, get_prompt_templates_industry, build_prompt_on_industry
from utils import filter_unfinished_output, filter_length_token, filter_length_sentence, select_selling_point, first_token_upper

In [ ]:
prompt = "Write a hook for product based on selling point.\n\
product:Coffee; selling points: quick and fast, tasty =>This is a must-have if you cannot live without coffee, especially if your time in the morning is tight, you need to watch this!\n\
product: Shake powder; selling points: provide all proteins and nutrients, taste good, many flavors =>I don't know about you, but I have been searching for so many products to finally find a proper protein shake. \n\
product: Cheesecake; selling points: low fat and sugar free, delicious, smooth, creamy, $10 =>Who says you can't have cheesecake on my diet? I highly recommend this cheesecake.\n"


prompt_no_sps = "Write a hook for product based on selling point.\n\
product:Coffee; selling points: =>This is a must-have if you cannot live without coffee, especially if your time in the morning is tight, you need to watch this!\n\
product: Shake powder; selling points: =>I don't know about you, but I have been searching for so many products to finally find a proper protein shake. \n\
product: Cheesecake; selling points: =>Who says you can't have cheesecake on my diet? I highly recommend this cheesecake.\n"



prompt_test = "Write a refined ad hook for product considering selling points.\n\
product: maltesers; selling point: crunchy, chewy =>Maltesers are a must if you want a delicious chocolate treat.\n\
product: Meal kit; selling point: fast and easy, nutritious =>Guys guess what I found for you! Only $10 for a meal, but very healthy and quick. Do you believe it?\n\
product: Chamomile Tea; selling point: no added sugar, contained ginger and chamoile =>Need to calm yourself before going to bed? Try this amazing chamomile tea!\n\
product: Goldfish Crackers; selling point: crunchy, always baked never fried =>"

prompt_test = "Write a refined ad hook for product considering selling points.\n\
Product: Pushup Bra\n\
Selling points: \n\
=>Hey girls have you ever experienced back rolls, side spillage, and irritating straps？This bra can help you solve those problems.\n\
Product: Men's suit\n\
Selling points: \n\
=>Here are 3 mistakes a guy makes.\n\
Product: Hoodies\n\
Selling points: \n\
=>Are you having a hard time to find something comfortable and pretty at the same time to wear at home?\n\
Product: Denim jeans\n\
Selling points: \n\
=>Hey bro, do you know you can style jeans with different tops to wear on all occasions?\n\
Product: Light Scarf\n\
Selling points: \n\
=>"

# product = 'Peanut Butter'
# sps = ''
# inputs = prompt_no_sps + 'product: {}; selling points:{} =>'.format(product, sps.lower())
inputs = prompt_test
print(inputs)
input_ids = tokenizer(inputs, return_tensors="pt").input_ids.to(device)
prompt_length = len(tokenizer.batch_decode(input_ids, skip_special_tokens=True)[0])
print(len(inputs), prompt_length)

In [ ]:
s_time = time.time()
# set_seed(777)
generated_ids = model.generate(input_ids, max_length=input_ids.shape[1]+35, min_length=12+input_ids.shape[1], \
                                do_sample=True, top_k=30, top_p=0.8, temperature=0.8, \
                                pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id, \
                                no_repeat_ngram_size=3, remove_invalid_values = True, num_return_sequences=5)
ori_gen_sent = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
gen_sent = [x[prompt_length:] for x in ori_gen_sent]
gen_sent = [x.split('\n')[0] for x in gen_sent]
for each in gen_sent:
    print(each)
print(time.time() - s_time)